In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from os import path
from run_snndensity import read_dataset,read_snnArray,dbscan
import sys
sys.path.append('../../')
from Evaluation.silhouette_coefficient import silhoutte_result
snn_nparray = read_snnArray('../snn_computation/snn_distance_metric.csv')


In [2]:
#DB SCAN

def demo(input_minpts,input_eps):
    data_labeled = dbscan(MinPts=input_minpts,Eps=input_eps,snn_nparray=snn_nparray)
    labels = data_labeled.labels_
    # print(labels)

# Remove outliers
    count = 0
    for item in labels:
        if item>-1:
            count = count+1
    # print("Number of true node ", count)  # Number of node after remove outliers

    new_distance_metric = np.zeros((count,count),dtype=float)
    label_dictionary_before = {}
    
    start = 0 
    for item in labels:
        label_dictionary_before[start]=item 
        start = start+1
    # print(label_dictionary_before)
    dictionary_no_outliers = {i:label_dictionary_before[i] for i in label_dictionary_before if label_dictionary_before[i]!=-1}
    # print(dictionary_no_outliers)

    k = dictionary_no_outliers.keys()

    # Create a new name for each node after remove outliers
    change_name_list={}
    start = 0
    for item in k:
        change_name_list[start] = item
        start = start+1

    #Add value to new snn_array, to use for calculate  Silhoutte - after remove outliers
    i=0
    j=0
    for i in range(0,count):
        for j in range(i+1,count):
            new_distance_metric[i][j] = snn_nparray[change_name_list[i]][change_name_list[j]]
            new_distance_metric[j][i] = new_distance_metric[i][j]

    # Silhoutte 
    new_label = list(dictionary_no_outliers.values())
    number_of_cluster = len(set(new_label)) 
    if len(set(new_label)) > 1:
        silhoute_removeouliers= silhoutte_result(new_distance_metric,new_label)
    else:
        print("Number of labels needs to > 1. ERROR!!")
        silhoute_removeouliers = -1

    return silhoute_removeouliers,len(new_label),number_of_cluster,label_dictionary_before


In [3]:
list_minpts = [i for i in range (3,20)]
eps = 0.65
print("Silhoutte \t Number of Clusters \t Number of nodes")
for item in list_minpts:
    score,number_of_nodes,num_clusters,label_dictionary = demo(item,eps)
    print(score, num_clusters,number_of_nodes)

Silhoutte 	 Number of Clusters 	 Number of nodes
0.5196751942481924 19 129
0.5100750005371928 12 108
0.517969882717201 7 88
0.508209032278478 6 83
0.5390059075669844 6 78
0.506701325683917 5 71
0.506701325683917 5 71
0.5003226645849598 4 61
0.5040896358543417 3 51
0.5040896358543417 3 51
0.5277210884353741 2 35
Number of labels needs to > 1. ERROR!!
-1 1 22
Number of labels needs to > 1. ERROR!!
-1 1 20
Number of labels needs to > 1. ERROR!!
-1 1 20
Number of labels needs to > 1. ERROR!!
-1 1 20
Number of labels needs to > 1. ERROR!!
-1 1 20
Number of labels needs to > 1. ERROR!!
-1 1 20


In [8]:
list_eps=np.arange(0.05, 1, 0.05).tolist()
for item in list_eps:
    score,number_of_nodes,num_clusters,label_dictionary = demo(2,item)
    print(item,':',score, num_clusters,number_of_nodes)





0.05 : 0.7932184353004214 20 72
0.1 : 0.756106650555935 24 93
0.15000000000000002 : 0.7691852630026038 27 103
0.2 : 0.7200716907872788 30 112
0.25 : 0.7550900773252771 32 120
0.3 : 0.7189566755760081 36 135
0.35000000000000003 : 0.691094855222317 38 144
0.4 : 0.6664600668535048 42 157
0.45 : 0.6568614967365756 46 167
0.5 : 0.6394588041227534 52 181
0.55 : 0.5918509261529006 55 196
0.6000000000000001 : 0.5742705145674987 61 210
0.6500000000000001 : 0.5437769251459266 68 227
0.7000000000000001 : 0.5216897677942888 78 248
0.7500000000000001 : 0.46860608827862343 96 293
0.8 : 0.34814728315897353 145 426
0.8500000000000001 : 0.20145674999047672 272 838
0.9000000000000001 : 0.09639488660281718 576 2014
0.9500000000000001 : 0.022828595481381782 509 4704


In [11]:
core,number_of_nodes,num_clusters,label_dictionary = demo(2,0.05)
print(score,number_of_nodes,num_clusters)
from collections import defaultdict
df = pd.read_csv('../../../TwitterDataset/DE/DE_target.csv')
df = df[['new_id','days','views']]
v = defaultdict(list)

for key, value in sorted(label_dictionary.items()):
    v[value].append(key)
for key in v.keys():
    if (key!=-1):
        print("Cluster: ",key)
        nodes = v[key]
        print(nodes)

0.022828595481381782 72 20
Cluster:  0
[1257, 1266, 1319, 7770, 7777, 7796, 7802, 7803, 7812, 7827, 7830, 7833, 7834, 7847, 7856, 7858, 7872, 7875]
Cluster:  1
[1430, 7644]
Cluster:  2
[3341, 3407, 3454]
Cluster:  3
[3679, 3740]
Cluster:  4
[3811, 4826]
Cluster:  5
[3852, 4783]
Cluster:  6
[4407, 9497]
Cluster:  7
[5557, 5562]
Cluster:  8
[5792, 5816]
Cluster:  9
[6007, 6027, 6099, 6129, 8261, 8295]
Cluster:  10
[6110, 6211, 6292]
Cluster:  11
[6497, 6504, 6529]
Cluster:  12
[6582, 6594]
Cluster:  13
[6698, 6703, 6712, 6750, 6774, 6776, 8990]
Cluster:  14
[6823, 8059]
Cluster:  15
[7748, 7757]
Cluster:  16
[8340, 8357]
Cluster:  17
[8400, 8432, 8435, 8462, 8558]
Cluster:  18
[8864, 8878, 8905]
Cluster:  19
[9201, 9218]


In [12]:


# Function to count How many nodes that every items in a list shared as neighbors

def count_sharing_nodes(input_list:list,path_knn=''):
    count = 0
    size = len(input_list)
    
    list_of_list=[]
    df = pd.read_csv(path_knn,header=FALSE)
    print(df)
    return count
a=[]
count_sharing_nodes(a,'../knn_metrics/knn_metric.csv')

     33 1124 4770 2471 4503 9134 1131 7260 2012 1863 2325 709 4144 484 1673 1967 2004 4631 1757 2432
0     3776 7482 1318 3890 8209 2058 4284 1878 5670 6...                                             
1     206 1647 7347 1146 1084 510 928 956 655 625 46...                                             
2     1129 4526 56 250 4893 201 131 964 1591 6109 86...                                             
3     1947 563 484 2257 1322 1463 2668 7818 1654 877...                                             
4     6592 6703 7607 3541 1367 7605 6862 8475 8476 8...                                             
...                                                 ...                                             
9492  7748 1430 7872 1319 7770 7777 7796 7802 7803 7...                                             
9493  7748 1430 7872 1319 7770 7777 7796 7802 7803 7...                                             
9494  7748 1430 7872 1319 7770 7777 7796 7802 7803 7...                                    

0